In [99]:
import pandas as pd
import re

In [100]:
def to_scientific_notation(value, decimal_places=4):
    """
    숫자를 지정된 소수점 자릿수의 지수 표기법으로 변환합니다.
    """
    format_string = f'{{:.{decimal_places}e}}'
    return format_string.format(value)

def convert_column_to_scientific(df, column_name, decimal_places=4):
    """
    DataFrame의 특정 열을 지수 표기법으로 변환합니다.
    """
    df[column_name] = df[column_name].apply(lambda x: to_scientific_notation(x, decimal_places))
    return df

In [101]:
# general information
file_path = './kinetic_DB/'
i_list = list(range(1,250))
#i_list = [101]

In [102]:
# bolsigdb.dat 생성
with open('bolsigdb.dat','w') as file:
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        f = convert_column_to_scientific(f, 'Energy(eV)')
        f = convert_column_to_scientific(f, 'Cross_section(m2)')

        type = f['Type'].loc[0]
        reaction = f['Reaction'].loc[0]
        if type == 'ELASTIC' or 'EFFECTIVE':
            eloss = str(f['ELoss'].loc[0])+' / mass ratio'
        else:
            eloss = str(f['ELoss'].loc[0])+' / threshold energy'

        line = '-----------------------------'

        file.write(type+'\n')
        file.write(reaction+'\n')
        file.write(eloss+'\n')
        file.write(line+'\n')
        for j in range(len(f)):
            file.write('  '+str(f['Energy(eV)'].loc[j])+'  '+str(f['Cross_section(m2)'].loc[j])+'\n')
        file.write(line+'\n')
        file.write('\n')
    
    file.close()

In [103]:
# kinep.inp 생성
with open('kinet.inp','w') as file:
    species = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        species += f['Formula'].loc[0].split(' ')
    
    f2 = pd.read_csv(file_path+"reaction_gas.csv")
    for i in range(len(f2)):
        species += f2['Reaction'].iloc[i].split(' ')
    species = list(set(species))
    species.remove('+')
    species.remove('->')
    species.remove('=>')
    species.remove('E')
    species.remove('e')
    species = ['e']+species

    if len(species) > 31:
        species1 = species[:31]
        species2 = species[31:]
    else:
        species1 = species
        species2 = []

    species1 = ' '.join(species1)
    species2 = ' '.join(species2)    

    file.write('ELEMENTS'+'\n')
    file.write('e C H'+'\n')
    file.write('END'+'\n')
    file.write('\n')
    file.write('SPECIES'+'\n')
    file.write(species1+'\n')
    if species2 != []:
        file.write(species2+'\n')
    file.write('END'+'\n')
    file.write('\n')

    bolsig = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        bolsig += f['Species'].loc[0].split(' ')
    bolsig = list(set(bolsig))
    bolsig.remove('/')
    bolsig.remove('e')
    bolsig = ' '.join(bolsig)
    bolsig = bolsig
    
    file.write('BOLSIG'+'\n')
    file.write(bolsig+'\n')
    file.write('END'+'\n')
    file.write('\n')

    file.write('REACTIONS'+'\n')

    file.write('#'+'\n')
    file.write('# Kinetic parameters'+'\n')
    file.write('#'+'\n')
    file.write('$ double precision, parameter :: R = 8.314d-3'+'\n')
    file.write('$ double precision, parameter :: fec = 1.000d0'+'\n')
    file.write('$ double precision, parameter :: fir = 1.000d0'+'\n')
    file.write('$ double precision, parameter :: fid = 1.000d0'+'\n')
    file.write('$ double precision, parameter :: fin = 1.000d0'+'\n')
    for i in range(134):
        para = 'f' + str(i)
        file.write('$ double precision, parameter :: ' + para + '= 1.000d0'+'\n')

    file.write('#'+'\n')
    file.write('# electron collision reaction'+'\n')
    file.write('#'+'\n')

    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        if f['Type'].loc[0] != 'ELASTIC' and f['Type'].loc[0] != 'EFFECTIVE':
            formula = f['Formula'].loc[0]
            formula = formula.replace('E','e').replace('->','=>')
            formula = formula
            bolsig_comp = formula.replace('e','').replace('+','').replace('=>','->')
            bolsig_comp = bolsig_comp.split(' ')
            bolsig_comp = [item for item in bolsig_comp if item != '']
            bolsig_comp = ' '.join(bolsig_comp)
            if f['Type'].loc[0] == 'IONIZATION':
                bolsig_comp += '+'
            bolsig_comp = bolsig_comp
            file.write(formula)
            file.write('\t'*3)
            file.write('! fec * Bolsig+ '+ bolsig_comp)
            file.write('\n')
    file.write('#'+'\n')

    file.write('# Ion recombination reaction'+'\n')
    file.write('#'+'\n')
    file.write('e + e + CH4^+ => e + CH4')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + CH3^+ => e + CH3')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + CH2^+ => e + CH2')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + CH^+ => e + CH')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C2H6^+ => e + C2H6')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C2H5^+ => e + C2H5')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C2H4^+ => e + C2H4')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C2H3^+ => e + C2H3')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C2H2^+ => e + C2H2')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C3H8^+ => e + C3H8')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C3H7^+ => e + C3H7')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C3H6^+ => e + C3H6')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C3H5^+ => e + C3H5')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + C3H4^+ => e + C3H4')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + H^+ => e + H')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('e + e + H2^+ => e + H2')
    file.write('\t'*3)
    file.write('! fir')
    file.write('\n')
    file.write('#'+'\n')
    file.write('# Ion dissociative reaction'+'\n')
    file.write('#'+'\n')
    for i in range(21):
        file.write(f2['Reaction'].iloc[i])
        file.write('\t'*3)
        file.write('! fid * ' + f2['Value'].iloc[i])
        file.write('\n')
    file.write('#'+'\n')
    file.write('# Ion-Neutral reaction'+'\n')
    file.write('#'+'\n')
    for i in range(97):
        file.write(f2['Reaction'].iloc[i+21])
        file.write('\t'*3)
        file.write('! fin * ' + f2['Value'].iloc[i+21])
        file.write('\n')
    file.write('#'+'\n')
    file.write('# Neutral-Neutral reaction'+'\n')
    file.write('#'+'\n')
    index = 0
    for i in range(len(f2)-118):
        file.write(f2['Reaction'].iloc[i+118])
        file.write('\t'*3)
        para = 'f' + str(index)
        file.write('! '+ para+ '* ' + f2['Value'].iloc[i+118])
        index += 1
        file.write('\n')

    file.write('END')

In [104]:
f2 = pd.read_csv(file_path+"reaction_gas.csv")

In [105]:
f2

,Unnamed: 0,Reaction,Value,Number
0,0,e + CH5^+ => CH3 + H + H,2.57d-07 * (300./Tgas)**0.3,NaN
1,1,e + CH5^+ => CH2 + H2 + H,6.61d-08 * (300./Tgas)**0.3,NaN
2,2,e + CH4^+ => CH3 + H,1.18d-08 * (300./Tgas)**0.5,NaN
3,3,e + CH4^+ => CH2 + H + H,2.42d-08 * (300./Tgas)**0.5,NaN
4,4,e + CH4^+ => CH + H2 + H,1.41d-08 * (300./Tgas)**0.5,NaN
...,...,...,...,...
247,247,C4H9H + C3H5 => C3H6 + C4H9,8.63d-14*(Tgas/298.)**3.30*exp(-83.06/(R*Tgas)),57.0
248,248,C4H9H + CH2 => C5H12,9.61d-13,94.0
249,249,C5H12 => CH3 + C4H9,3.16d16*exp(-331./(R*Tgas)),105.0
250,250,H2 => H + H,1.88d-8*(Tgas/298.)**(-1.10)*exp(-437./(R*Tgas)),106.0
